In [21]:
import pandas as pd
import numpy as np
import random as rnd
import os
from sklearn import preprocessing

In [22]:
## Fill missing values with mean/mode

from sklearn.base import TransformerMixin

class meanModeImputer(TransformerMixin):

    def __init__(self):
        pass
        
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [23]:
# GKnn Algorithm
from numpy import mean
from statistics import mode
import time

def GKnn(impute_df):
    
    for i in range(len(empty_loc_cols)-1):

        ## Differentiate numerical and categorical data
        numerical_cols = impute_df._get_numeric_data().columns #fetches just the numeric columns
        cols = impute_df.columns                                 # entire columns in dataset
        categorical_cols = list(set(cols) - set(numerical_cols)) # gives us just the categorical columns
        num_df = impute_df.loc[:,numerical_cols]                 #locates all rows in numerical dataframe column by their labels
        cat_df = impute_df.loc[:,categorical_cols]               #locates all columns in categorical dataframe column by their labels
        
        
        num_ck=num_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of emptyelements in a numeric DF
        maxIter = 2
        num_cp=num_df.copy()
        num_cp.drop([i + 1], axis = 0)                           #dropping adjacent emptyrows on indexing for numerical dataset to avoid duplication

        cat_ck=cat_df.iloc[empty_loc_cols[i],:]                  # gets the location/index of empty elements in a categorical DF
        cat_cp=cat_df.copy()
        cat_cp.drop([i + 1], axis = 0)                           # dropping adjacent emptyrows on indexing for categorical dataset to avoid duplictaion
        
        
        def greyRelationalGrade(impute_df):
           
            t1=pd.DataFrame() #initializing a dataframe
            t2=pd.DataFrame(data = None, columns=cat_cp.columns, index=cat_cp.index) #has the shape of the cat_cp dataframe
            
            for j in range(num_cp.index.size):
                temp=pd.Series(num_cp.iloc[j,:]-num_ck)
                t1=t1.append(temp,ignore_index=True)
                
                if(cat_cp.iloc[j,:].equals(cat_ck)):
                    t2.iloc[j,:] = 1
                else:
                    t2.iloc[j,:] = 0

            t1.reset_index(drop=True, inplace=True)
            t2.reset_index(drop=True, inplace=True)
            t = pd.concat([t1, t2], axis=1)

                
            mmax=t.abs().max().max()
            mmin=t.abs().min().min()
            rho=0.5

            # Computing grey correlation coefficient
            GRC=((mmin+rho*mmax)/(abs(t)+rho*mmax))

            # Computing the GRG and getting the rank
            GRG=GRC.sum(axis=1)/GRC.columns.size
            GRGSort = GRG.sort_values(ascending= False)
            
            return GRGSort
        
        count = 0
        previous = pd.DataFrame(data=incomplete_df.iloc[empty_loc_cols])
        prev = previous.mean().sum()
        for j in range(len(empty_loc[0])-1):
            startTime = time.time()
            result= greyRelationalGrade(impute_df)
            resultSorted = result.sort_values(ascending = False)
            neighbours =  resultSorted.iloc[0:7].index
            # find the most count of the values at this index of these instances
            values  = impute_df.iloc[neighbours]
            num_neigh = values._get_numeric_data().columns
            cat_neigh = list(set(values) - set(num_neigh))
            num_neigh_cols = values.loc[:, num_neigh]
            cat_neigh_cols = values.loc[:,cat_neigh] 
            nMode = cat_neigh_cols.value_counts().index[1]
            #imputation using k nearest neighbor instances
            if(empty_loc_cols[i] == empty_loc[0][j]):
                impute_df.iloc[empty_loc[i][j]] = nMode[(len(empty_loc_col)-1)]

            count += 1      
            endTime = time.time()
            tol = 10e-4     
            ## convergence criteria
            if count >= maxIter or abs(impute_df.iloc[empty_loc_cols].mean().sum() - prev) < tol  : 
                break
            prev = impute_df.iloc[empty_loc_cols].mean().sum()
        
            
    return impute_df

In [24]:
## import xlsx files as pandas dataframe
pathIncomplete = '/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Incomplete Datasets Without Labels/Final Demo Cat'
complete_df = pd.read_excel('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Original Datasets Without Labels/TTTTEG.xlsx', header = None)
incomplete_df_name_list = []
incomplete_df_list = []
for filename in os.listdir(pathIncomplete):
    if filename.endswith('.xlsx'):
        incomplete_df_list.append(pd.read_excel(pathIncomplete+"/"+filename, header=None))
        incomplete_df_name_list.append(filename.split(".")[0])

In [25]:
os.listdir(pathIncomplete)

['TTTTEG_AW_10.xlsx', 'TTTTEG_C_5.xlsx']

In [26]:
#Using AE since it is Categorical
imputed_df_list = []
ae_data = {}
index = 0
for incomplete_df in incomplete_df_list:

    incomplete_df.reset_index()
    complete_df.reset_index()
    
    complete_df.columns=["Col"+str(i) for i in range(complete_df.shape[1])] #added header to the columns
    incomplete_df.columns=["Col"+str(i) for i in range(incomplete_df.shape[1])] #added header to the columns
    
    ## find empty locations
    empty_loc = np.where(pd.isnull(incomplete_df))
    empty_loc_cols = np.unique(empty_loc[0])
    empty_loc_col = np.unique(empty_loc[1])
    
    ## Replace NaN values with mean/mode values
    incomplete_df = meanModeImputer().fit_transform(incomplete_df)
    
    ## GkNN
    imputed_df = GKnn(incomplete_df)
    
    ## Compute AE
    imputed_ae = 0;
    for x in range(len(empty_loc[0]) - 1):
        if(imputed_df.iloc[empty_loc[0][x], empty_loc[1][x]] == complete_df.iloc[empty_loc[0][x], empty_loc[1][x]]):
            imputed_ae+=1
    AE = (complete_df.size - len(empty_loc[0]) + imputed_ae) / complete_df.size
    ae_data.update( {incomplete_df_name_list[index]: AE} )
    
    
    imputed_df_list.append(imputed_df)
    index+=1
ae_data


{'TTTTEG_AW_10': 0.9458362328926003, 'TTTTEG_C_5': 0.9715843191834841}

In [27]:
#save imputed dataframes to xlsx
index = 0
for df in imputed_df_list:
    df.to_excel("/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/HOV_K7/"+incomplete_df_name_list[index]+".xlsx")
    index+=1

In [28]:
# Download Dictionary as csv file
import csv
with open('/Users/lilianude/Documents/UDE LILIAN/GENG 8900 Data Mining/Datasets/Graph/NRMS_AE/HOV_K7.csv', 'w') as csv_file:  
    writeTo = csv.writer(csv_file)
    for key, value in ae_data.items():
        writeTo.writerow([key, value])

In [29]:
ae_data

{'TTTTEG_AW_10': 0.9458362328926003, 'TTTTEG_C_5': 0.9715843191834841}

In [30]:
imputed_df_list

[    Col0 Col1 Col2 Col3 Col4 Col5 Col6 Col7 Col8
 0      x    x    x    x    o    o    x    o    o
 1      x    x    x    x    x    x    x    x    x
 2      x    x    x    x    x    x    o    o    x
 3      x    x    x    x    x    x    o    b    b
 4      x    x    x    x    x    o    x    o    x
 ..   ...  ...  ...  ...  ...  ...  ...  ...  ...
 953    o    x    x    x    o    o    o    x    x
 954    o    x    o    x    x    o    x    o    x
 955    x    x    o    x    x    x    x    o    x
 956    o    x    o    o    x    x    x    o    x
 957    o    o    x    x    x    o    o    x    x
 
 [958 rows x 9 columns],
     Col0 Col1 Col2 Col3 Col4 Col5 Col6 Col7 Col8
 0      x    x    x    x    x    x    x    x    x
 1      x    x    x    x    o    o    o    x    o
 2      x    x    x    x    o    o    o    o    x
 3      x    x    x    x    o    o    o    b    b
 4      x    x    x    x    o    o    b    o    b
 ..   ...  ...  ...  ...  ...  ...  ...  ...  ...
 953    o    x    x    